In [35]:
import numpy as np
from scipy.fftpack import fft, fftshift, ifft, fftfreq
from scipy.signal import freqz, firwin
import matplotlib.pyplot as plt
import csv
import math
import binascii

In [28]:
with open('gr1channel.csv', 'r') as file:
    FIR = file.read().replace('\n', '')
FIR = [float(x) for x in FIR.split()] 

with open('gr1file.csv', 'r') as file:
    data = file.read().replace('\n', '')
data = [float(x) for x in data.split()]

In [29]:
d = len(data)/1056

# removing cyclic prefix
no_prefix = []
for i in range(int(d)):
    no_prefix.append(data[i*1056 + 32:(i+1)*1056])

In [30]:
# Function to display something quickly
def plot_y(y):
    x = np.linspace(0, len(y), len(y))
    plt.plot(x, y)
    plt.show

In [56]:
freq = fft(FIR, n = 1024)[1:512]
binary = []
for i in range(len(no_prefix)):
    fft_data = fft(no_prefix[i])[1:512]
    
    for j in range(len(fft_data)):
        div = fft_data[j]/freq[j]
        
        bit0 = 0
        bit1 = 0
        
        if div.real < 0:
            bit1 = 1
        if div.imag < 0:
            bit0 = 1
            
        binary.append(bit0)
        binary.append(bit1)

bin_single_sting = ""
for i in range(len(binary)):
    bin_single_sting += str(binary[i])

In [78]:
def str_to_bytearray(string_data):
    new_data = []
    for i in range(0, len(string_data), 8):
        new_data.append(string_data[i:i+8])  

    int_data = [] 
    for i in new_data:
        int_data.append(int(i,2))

    return bytearray(int_data)

In [79]:
# Find information about the file name and the byte length of the file
file_seperation = []
count = 0
for i in range(math.floor(len(binary)/8)):
    byte = binary[i*8:(i+1)*8]
    zero = True
    for bit in byte:
        if bit == 1:
            zero = False
            
    if zero:
        #print(byte, "zero")
        file_seperation.append(i*8)
            
        count += 1
    
    if count == 2:
        break

In [89]:
# Extract file name and length of the file
Filename= str_to_bytearray(bin_single_sting[0:file_seperation[0]]).decode()
Length = int(str_to_bytearray(bin_single_sting[file_seperation[0]+8:file_seperation[1]]).decode())

extra_bits = len(binary) - file_seperation[1] - 8 - Length*8

raw_file = bin_single_sting[file_seperation[1] + 8 : -1*extra_bits]

newFile = open("key.bmp", "wb")
newFile.write(str_to_bytearray(raw_file))

28938